最新版！

In [2]:
using CUDA
using MAT
using LinearAlgebra
using Random
using Statistics
using NPZ
using Printf

# ==== 模型参数 ====
g_to_kg = 1e-3                      # g → kg
kgm3_to_ppb = 1.529e7              # kg/m³ → ppb（甲烷）

u      = 2.5f0                     # 风速 m/s
Ry, ry = 0.469f0, 0.903f0
Rz, rz = 0.017f0, 1.380f0

domain_size = 40000               # m
resolution = 1                    # m
nx = domain_size ÷ resolution     # = 40000
ny = nx

subgrid_size_m = 4000
subgrid_size_px = subgrid_size_m ÷ resolution  # = 4000
n_subgrid = domain_size ÷ subgrid_size_m       # = 10

# ==== CUDA 核函数 ====
function kernel_total_conc_add!(
    C, nx, ny, dx, dy,
    sources_x, sources_y, sources_q,
    u, Ry, ry, Rz, rz
)
    ix = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    iy = (blockIdx().y - 1) * blockDim().y + threadIdx().y
    if ix > nx || iy > ny
        return
    end

    xpos = (ix - 1) * dx
    ypos = (iy - 1) * dy

    c = 0.0f0
    ns = length(sources_x)
    @inbounds for k in 1:ns
        sx = sources_x[k]
        sy = sources_y[k]
        q  = sources_q[k]

        δy = sy - ypos
        if δy <= 0f0
            continue
        end

        σy = Ry * δy^ry
        σz = Rz * δy^rz

        term1 = q / (π * u * σy * σz)
        term2 = exp(-((xpos - sx)^2) / (2f0 * σy^2))
        c += term1 * term2
    end

    C[iy, ix] += c  # 保持行是 y，列是 x，更符合图像坐标和空间直觉
    
    return nothing
end

# ==== 构建源列表 ====
function build_sources(source_map::Matrix{Float32}; offset_x=0f0, offset_y=0f0)
    sx_list = Float32[]
    sy_list = Float32[]
    sq_list = Float32[]

    @inbounds for j in 1:size(source_map, 2), i in 1:size(source_map, 1)
        q_gps = source_map[i, j]
        if q_gps == 0f0
            continue
        end
        q = q_gps * g_to_kg                    # g/s → kg/s
        push!(sx_list, (i - 1 + rand(Float32)) + offset_x)
        push!(sy_list, (j - 1 + rand(Float32)) + offset_y)
        push!(sq_list, q)
    end

    return (sx_list, sy_list, sq_list)
end

# ==== CUDA 并行模拟 ====
function simulate_and_save(C::CuArray{Float32}, sx, sy, sq, res, fname)
    threads = (16, 16)
    blocks = (cld(size(C,1), threads[1]), cld(size(C,2), threads[2]))
    batch_size = 1_000_000
    total_sources = length(sx)
    total_batches = ceil(Int, total_sources / batch_size)

    for batch in 1:total_batches
        range = (batch-1)*batch_size+1 : min(batch*batch_size, total_sources)
        sources_x = CuArray(sx[range])
        sources_y = CuArray(sy[range])
        sources_q = CuArray(sq[range])

        @cuda threads=threads blocks=blocks kernel_total_conc_add!(
            C, size(C,1), size(C,2), Float32(res), Float32(res),
            sources_x, sources_y, sources_q,
            u, Ry, ry, Rz, rz
        )
        synchronize()
    end

    C_cpu = Array(C)
    C_ppb = C_cpu .* kgm3_to_ppb

    matopen(fname, "w") do f
        write(f, "C", C_ppb)
    end
end

# ==== 主函数 ====
function run_simulation(input_path::String)
    
    output_dir = "/root/autodl-tmp/result3"
    
    println("📂 加载源强图文件: $input_path")
    full_map = npzread(input_path)
    @assert size(full_map) == (nx, ny) "❌ 输入尺寸应为 $(nx)x$(ny)"

    println("▶️ 1. 全图模拟 ...")
    sx, sy, sq = build_sources(full_map)
    C_full = CUDA.zeros(Float32, nx, ny)
    simulate_and_save(C_full, sx, sy, sq, resolution,
        joinpath(output_dir, replace(basename(input_path), ".npy" => "_fullsim.mat")))
    println("✅ 全图模拟完成。")
    
    #println("▶️ 2. 局部区块模拟 ...")
    #for i in 0:n_subgrid-1, j in 0:n_subgrid-1
        #x_start = i * subgrid_size_px + 1
        #x_end   = (i+1) * subgrid_size_px
        #y_start = j * subgrid_size_px + 1
        #y_end   = (j+1) * subgrid_size_px

        #submap = full_map[x_start:x_end, y_start:y_end]
        #if all(submap .== 0f0)
            #continue
        #end

        #println("🔍 非零区块: ($i, $j)，执行局部模拟 ...")
        #offset_x = Float32(x_start - 1)
        #offset_y = Float32(y_start - 1)
        #sx, sy, sq = build_sources(submap; offset_x=offset_x, offset_y=offset_y)

        #C_local = CUDA.zeros(Float32, subgrid_size_px, subgrid_size_px)
        #output_name = joinpath(output_dir, @sprintf("%s_block_x%02d_y%02d.mat",
                #replace(basename(input_path), ".npy" => ""), i, j))
        #simulate_and_save(C_local, sx, sy, sq, resolution, output_name)
    #end
    #println("✅ 所有局部区块模拟完成。")
end

run_simulation("/root/autodl-tmp/output_sources/400m.npy")

📂 加载源强图文件: /root/autodl-tmp/output_sources/400m.npy
▶️ 1. 全图模拟 ...
✅ 全图模拟完成。


双精度

In [ ]:
using CUDA
using MAT
using LinearAlgebra
using Random
using Statistics
using NPZ
using Printf

# ==== 模型参数 ====
g_to_kg = 1e-3
kgm3_to_ppb = 1.529e7

u      = 2.5
Ry, ry = 0.306, 0.885
Rz, rz = 0.072, 1.021

domain_size = 40000
resolution = 1
nx = domain_size ÷ resolution
ny = nx

subgrid_size_m = 4000
subgrid_size_px = subgrid_size_m ÷ resolution
n_subgrid = domain_size ÷ subgrid_size_m

# ==== CUDA Kernel ====
function kernel_total_conc_add!(
    C, nx, ny, dx, dy,
    sources_x, sources_y, sources_q,
    u, Ry, ry, Rz, rz
)
    ix = (blockIdx().x - 1) * blockDim().x + threadIdx().x
    iy = (blockIdx().y - 1) * blockDim().y + threadIdx().y
    if ix > nx || iy > ny
        return
    end

    xpos = (ix - 1) * dx
    ypos = (iy - 1) * dy

    c = 0.0
    ns = length(sources_x)
    @inbounds for k in 1:ns
        sx = sources_x[k]
        sy = sources_y[k]
        q  = sources_q[k]

        δy = sy - ypos
        if δy <= 0.0
            continue
        end

        σy = Ry * δy^ry
        σz = Rz * δy^rz

        term1 = q / (π * u * σy * σz)
        term2 = exp(-((xpos - sx)^2) / (2.0 * σy^2))
        c += term1 * term2
    end

    C[ix, iy] += c
end

# ==== 构建源 ====
function build_sources(source_map::Matrix{Float64}; offset_x=0.0, offset_y=0.0)
    sx_list = Float64[]
    sy_list = Float64[]
    sq_list = Float64[]

    @inbounds for j in 1:size(source_map, 2), i in 1:size(source_map, 1)
        q_gps = source_map[i, j]
        if q_gps == 0.0
            continue
        end
        q = q_gps * g_to_kg  # g/s → kg/s
        push!(sx_list, (i - 1 + rand()) + offset_x)
        push!(sy_list, (j - 1 + rand()) + offset_y)
        push!(sq_list, q)
    end

    return (sx_list, sy_list, sq_list)
end

# ==== 并行模拟 ====
function simulate_and_save(C::CuArray{Float64}, sx, sy, sq, res, fname)
    threads = (16, 16)
    blocks = (cld(size(C,1), threads[1]), cld(size(C,2), threads[2]))
    batch_size = 1_000_000
    total_sources = length(sx)
    total_batches = ceil(Int, total_sources / batch_size)

    for batch in 1:total_batches
        range = (batch-1)*batch_size+1 : min(batch*batch_size, total_sources)
        sources_x = CuArray(sx[range])
        sources_y = CuArray(sy[range])
        sources_q = CuArray(sq[range])

        @cuda threads=threads blocks=blocks kernel_total_conc_add!(
            C, size(C,1), size(C,2), res, res,
            sources_x, sources_y, sources_q,
            u, Ry, ry, Rz, rz
        )
        synchronize()
    end

    C_cpu = Array(C)
    C_ppb = C_cpu .* kgm3_to_ppb

    matopen(fname, "w") do f
        write(f, "C", C_ppb)
    end
end

# ==== 主程序入口 ====
function run_simulation(input_path::String)
    println("📂 加载源强图文件: $input_path")
    full_map = convert(Matrix{Float64}, npzread(input_path))
    @assert size(full_map) == (nx, ny) "❌ 输入尺寸应为 $(nx)x$(ny)"
    
    println("▶️ 1. 全图模拟 ...")
    sx, sy, sq = build_sources(full_map)
    C_full = CUDA.zeros(Float64, nx, ny)
    simulate_and_save(C_full, sx, sy, sq, resolution,
        joinpath(output_dir, replace(basename(input_path), ".npy" => "_fullsim.mat")))
    println("✅ 全图模拟完成。")

    println("▶️ 2. 局部区块模拟 ...")
    for i in 0:n_subgrid-1, j in 0:n_subgrid-1
        x_start = i * subgrid_size_px + 1
        x_end   = (i+1) * subgrid_size_px
        y_start = j * subgrid_size_px + 1
        y_end   = (j+1) * subgrid_size_px

        submap = full_map[x_start:x_end, y_start:y_end]
        if all(submap .== 0.0)
            continue
        end

        println("🔍 非零区块: ($i, $j)，执行局部模拟 ...")
        offset_x = x_start - 1
        offset_y = y_start - 1
        sx, sy, sq = build_sources(submap; offset_x=offset_x, offset_y=offset_y)
        
        C_local = CUDA.zeros(Float64, subgrid_size_px, subgrid_size_px)
        output_name = joinpath(output_dir, @sprintf("%s_block_x%02d_y%02d.mat",
                replace(basename(input_path), ".npy" => ""), i, j))
        simulate_and_save(C_local, sx, sy, sq, resolution, output_name)
        
    end
    println("✅ 所有局部区块模拟完成。")
end